### NYC Arrests - Crime

<hr>

This notebook extracts NYC arrest data from NYC OpenData. Data is then processed and exported to a CSV file which is then imported into Tableau for data visualization.

Before running this notebook, ensure you have the following shape files:
<ul>
   <li>Neighborhood Tabluation Areas (NTA)</li>
   <li>Police Precincts</li> 
<ul>

<hr>

### Imports

In [1]:
#!/usr/bin/env python

# make sure to install necessary packages before running
# GeoPandas might be a bit difficult to install

import os
import datetime
import sys
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sodapy import Socrata

import geopandas
from shapely.geometry import Point

C:\Users\alexc\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Get data from NYC OpenData

In [2]:
socrata_domain = 'data.cityofnewyork.us'

# NYPD Arrest Data (Year to Date)
socrata_dataset_identifier_ny_data_ytd = 'uip8-fykc'

# NYPD Arrest Data (Historic)
socrata_dataset_identifier_ny_data_his = '8h9b-rp9u'

# read application token
with open('app-token.txt', 'r') as fp:
    contents = fp.read()
socrata_token = contents

# connect to data
client = Socrata(socrata_domain, socrata_token)

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
#client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata('data.cityofnewyork.us',
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
# results = client.get(socrata_dataset_identifier, limit=1000)

# Convert to pandas DataFrame
# results_df = pd.DataFrame.from_records(results)

#### Metadata associated with the data pull

In [ ]:
# Metadata
# metadata = client.get_metadata(socrata_dataset_identifier_ny_data_ytd)
# metadata_columns = [x['name'] for x in metadata['columns']]
# metadata_columns

In [ ]:
# Metadata for a particular column
# meta_arrest_col_date = [x for x in metadata['columns'] if x['name'] == 'ARREST_DATE'][0]
# meta_arrest_col

#### Get date range of the data and size of data

In [ ]:
# get date range of data
# note: string is returned
# most_recent_date = meta_arrest_col_date['cachedContents']['largest']
# oldest_date = meta_arrest_col_date['cachedContents']['smallest']
# num_of_results = int(meta_arrest_col_date['cachedContents']['not_null'])

# print("Most recent date: " + most_recent_date)
# print("Oldest date: " + oldest_date)
# print("Total number of not null results: "+ str(num_of_results))

### Get the data using the API

In [3]:
# Get data from start_date to end of previous calendar year
start_date = '2017-01-01'

# loop through the results (to prevent data from crashing)
loop_size = 10000

data = []
results = True
i = 0
while(results):
    results = client.get(socrata_dataset_identifier_ny_data_his, 
#                          where="ARREST_DATE >= '2019-01-01T00:00:00.000'",
                         where = f"ARREST_DATE >= '{start_date}T00:00:00.000'",
                         # select="column1,column2,etc",
                         # order="column1 ASC",
                         # can also do other operations 
                         limit=loop_size,
                         offset=loop_size * i)
    i += 1
    data.extend(results)

In [5]:
# Get data from year to date

now = datetime.datetime.now()
current_year =  now.year

results = True
i = 0
while(results):
    results = client.get(socrata_dataset_identifier_ny_data_ytd, 
                         where= f"ARREST_DATE >= '{current_year}-01-01T00:00:00.000'",
                         limit=loop_size,
                         offset=loop_size * i)
    i += 1
    data.extend(results)

In [7]:
# put the data in a pandas dataframe
results_df = pd.DataFrame(data)

### Quick exploration of data

In [ ]:
# # What is law_cat_cd with value 'I'? Likely Traffic Infraction
# results_df.loc[results_df['law_cat_cd'] == 'I'].head()

In [ ]:
# # quick summary of data
# results_df.describe()

### Clean data to make analysis easier

In [9]:
# change arrest_boro abbreviation to long form
boro_dict = {
    'B': 'Bronx',
    'S': 'Staten Island',
    'K': 'Brooklyn',
    'M': 'Manhattan',
    'Q': 'Queens',
}
results_df['arrest_boro'] = results_df.arrest_boro.replace(boro_dict)

# change perp_sex abbreviation to long form
perp_sex_dict = {
    'F': 'Female',
    'M': 'Male',
}
results_df['perp_sex'] = results_df.perp_sex.replace(perp_sex_dict)

# change law_cat_cd abbreviation to long form
law_cat_cd_dict = {
    'F': 'Felony',
    'M': 'Misdemeanor',
    'V': 'Violation',
    'I': 'Traffic Infraction',
}
results_df['law_cat_cd'] = results_df['law_cat_cd'].replace(law_cat_cd_dict)

In [10]:
# change perp_race from all caps to title case
results_df['perp_race'] = results_df['perp_race'].apply(lambda x: x.title())

# change perp_race from all caps to title case
results_df['ofns_desc'] = results_df['ofns_desc'].astype(str)
results_df['ofns_desc'] = results_df['ofns_desc'].apply(lambda x: x.title())

In [11]:
# change latitude and longitude from string to numeric
results_df['latitude'] = pd.to_numeric(results_df['latitude'])
results_df['longitude'] = pd.to_numeric(results_df['longitude'])
# print(results_df.dtypes)

### Prepare data for spatial analysis
##### Note: this step may take a long time

In [12]:
# add a column of the neighborhood name within NYC (for Tableau)

url = r"Data\Spatial Files\Neighborhood Tabulation Areas (NTA)\geo_export_0c82a76e-3045-414c-9b45-6c529ffc990f.shp"
gdf_locations = geopandas.read_file(url)

# convert to GeoDataFrame and create a column of shapely points
gdf = geopandas.GeoDataFrame(
    results_df, geometry=geopandas.points_from_xy(x=results_df.longitude, y=results_df.latitude)
)

def get_ntaname(shapely_point):
    """Returns the neighborhood name in NYC given a Shapely Point"""
    filter_ = gdf_locations['geometry'].contains(shapely_point)
    ntaname_pd_series = gdf_locations.loc[filter_, 'ntaname']
    if(len(ntaname_pd_series)==0):
        return "Unknown"
    else:
        ntaname = ntaname_pd_series.to_string(index=False).strip()
        return ntaname

# create a new column with the nyc neighborhood
# is there a faster way to do this? vectorize?
gdf['neighborhood'] = gdf['geometry'].map(get_ntaname)

In [13]:
# unknown neighborhoods (remove in Tableau)
unknown_neighborhoods_df = gdf[gdf['neighborhood'] == "Unknown"]

print(unknown_neighborhoods_df.shape)
# print(unknown_neighborhoods_df)

(143, 22)


In [ ]:
# results_df.head()

In [14]:
### Drop unnecessary columns
results_df.drop(columns=['arrest_key', 'x_coord_cd', 'y_coord_cd', 'geometry'], inplace=True)

### Summary of data and some data visualization

In [ ]:
# # unique arrest_boro
# print(results_df.arrest_boro.unique())

# # unique perp_sex
# print()
# print(results_df.perp_sex.unique())

# # unique perp_race
# print()
# print(results_df.perp_race.unique())

# # unique perp_race
# print()
# print(results_df['law_cat_cd'].value_counts(dropna=False))

# # unique pd_desc
# print()
# unique_pd_desc = results_df.pd_desc.unique()
# print(len(unique_pd_desc))

# print()
# print(results_df['arrest_boro'].value_counts(dropna=False))

# print()
# print(results_df['jurisdiction_code'].value_counts(dropna=False))


# print()
# print(results_df['neighborhood'].value_counts(dropna=False))

# print()
# print(results_df.shape)

# print()
# print(results_df.dtypes)

In [ ]:
# def order_labels(age_group_labels):
#     """Returns age_group label sorted in ascending order"""
#     ordered_labels = sorted(list(age_group_labels))
#     last_item = ordered_labels.pop()
#     ordered_labels.insert(0, last_item)
#     return ordered_labels
    
# age_group_labels = order_labels(results_df.age_group.unique())

# sns.countplot(results_df['age_group'], order=age_group_labels)

In [ ]:
# sns.countplot(results_df['perp_sex'], )

In [ ]:
# plot_race = sns.countplot(results_df['perp_race'], )
# tick_labels = plot_race.get_xticklabels()
# plot_race = plot_race.set_xticklabels(tick_labels, rotation=90)

In [ ]:
# sns.countplot(results_df['arrest_boro'], )

In [15]:
# export Pandas DataFrame to csv
today = datetime.date.today()
results_df.to_csv(f"Archived Files/{today}_nypd-arrests.csv", index_label="unique_id")